In [6]:
import torch
from model import *
import os
import numpy as np
from onnxruntime.quantization import quantize_static, QuantType


### Model PTQ

In [2]:
model = OneEE()
state = torch.load('trained_model.pth')
model.load_state_dict(state)

/tmp/ipykernel_15674/4280291455.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load('trained_model.pth')


<All keys matched successfully>

In [3]:
class CalibrationDataReader:
    def __init__(self, calibration_dir):
        self.calibration_files = [os.path.join(calibration_dir, f) for f in os.listdir(calibration_dir) if f.endswith('.npy')]
        self.data_index = 0

    def get_next(self):
        if self.data_index < len(self.calibration_files):
            input_data = np.load(self.calibration_files[self.data_index])
            input_data = np.expand_dims(input_data, axis=0)  # Add batch dimension
            self.data_index += 1
            return {'input': input_data.astype(np.float32)}
        else:
            return None

    def rewind(self):
        self.data_index = 0


In [4]:
calibration_dir = '../../../calibration_data'
calibration_data_reader = CalibrationDataReader(calibration_dir)

In [7]:
quantize_static(
    'trained_model.onnx',                 
    'trained_model_ptq.onnx',            
    calibration_data_reader,              
    quant_format=QuantType.QInt8,         
    weight_type=QuantType.QInt8           
)


### Size comparison

In [8]:
model_original = os.path.getsize('trained_model.onnx')/(1024**2)
model_ptq = os.path.getsize('trained_model_ptq.onnx')/(1024**2)

In [10]:
print(f"Original model size: {model_original:.2f} MB")
print(f"Post-training quantized model size: {model_ptq:.2f} MB")

Original model size: 650.88 MB
Post-training quantized model size: 162.75 MB
